In [1]:
import os
import sys
sys.path.append("./transformer/")
import tensorflow as tf
%load_ext tensorboard
from transformer import Transformer
from preprocess.batch_generator import BatchGenerator

/Users/hiroki/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/hiroki/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/hiroki/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/hiroki/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework

In [2]:
data_path = "data/natsume.txt"

In [3]:
batch_generator = BatchGenerator(spm_model_path="transformer/preprocess/spm_natsume.model")
batch_generator.load(data_path)

In [4]:
vocab_size = batch_generator.vocab_size

In [5]:
graph = tf.Graph()
with graph.as_default():
    transformer = Transformer(
        vocab_size=vocab_size,
        hopping_num=4,
        head_num=8,
        hidden_dim=512,
        dropout_rate=0.1,
        max_length=50,
    )
    transformer.build_graph()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Instructions for updating:
Use `tf.cast` instead.





Instructions for updating:
Use `tf.cast` instead.


In [6]:
save_dir = 'tmp/learning/transformer/'
log_dir = os.path.join(save_dir, 'log')
ckpt_path = os.path.join(save_dir, 'checkpoints/model.ckpt')

os.makedirs(log_dir, exist_ok=True)

In [7]:
with graph.as_default():
    global_step = tf.train.get_or_create_global_step()

    learning_rate = tf.placeholder(dtype=tf.float32, name='learning_rate')
    optimizer = tf.train.AdamOptimizer(
        learning_rate=learning_rate,
        beta2=0.98,
    )
    optimize_op = optimizer.minimize(transformer.loss, global_step=global_step)

    summary_op = tf.summary.merge([
        tf.summary.scalar('train/loss', transformer.loss),
        tf.summary.scalar('train/acc', transformer.acc),
        tf.summary.scalar('train/learning_rate', learning_rate),
    ], name='train_summary')
    summary_writer = tf.summary.FileWriter(log_dir, graph)
    saver = tf.train.Saver()

In [8]:
max_step = 100000
batch_size = 128
max_learning_rate = 0.0001
warmup_step = 4000

In [9]:
def get_learning_rate(step: int) -> float:
    rate = min(step ** -0.5, step * warmup_step ** -1.5) / warmup_step ** -0.5
    return max_learning_rate * rate

In [10]:
with graph.as_default():
    sess = tf.Session()
    sess.run(tf.global_variables_initializer())
    step = 0

In [11]:
with graph.as_default():
    for batch in batch_generator.get_batch(batch_size=batch_size):
        print(batch)
        feed = {
            **batch,
            learning_rate: get_learning_rate(step + 1),
        }
        _, loss, acc, step, summary = sess.run([optimize_op, transformer.loss, transformer.acc, global_step, summary_op], feed_dict=feed)
        summary_writer.add_summary(summary, step)

        if step % 100 == 0:
            print(f'{step}: loss: {loss},\t acc: {acc}')
            saver.save(sess, ckpt_path, global_step=step)

{'transformer/encoder_input:0': array([[   7,  448,   16, ...,    0,    0,    0],
       [ 128,    5,  571, ...,    0,    0,    0],
       [   7, 3876,   10, ...,    0,    0,    0],
       ...,
       [   7, 1215,  240, ...,    0,    0,    0],
       [   7, 2023,  921, ...,    0,    0,    0],
       [1462,  545, 1591, ...,    0,    0,    0]], dtype=int32), 'transformer/decoder_input:0': array([[   1,    7, 5201, ...,    0,    0,    0],
       [   1, 2378,  129, ...,    0,    0,    0],
       [   1,  311, 5126, ...,    0,    0,    0],
       ...,
       [   1,   14, 6087, ...,    0,    0,    0],
       [   1,    7,  980, ...,    0,    0,    0],
       [   1,   87,  281, ...,    0,    0,    0]], dtype=int32), 'transformer/is_training:0': True}


KeyboardInterrupt: 